# Choose Model Llama 3.2 with Fine-Tuning and Lora Layers

In [1]:
import torch
print("CUDA Available: ", torch.cuda.is_available())
print("CUDA Device Name: ", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# Verificar se CUDA está disponível para acelerar o processamento
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

CUDA Available:  True
CUDA Device Name:  NVIDIA GeForce RTX 3050 Ti Laptop GPU
Usando dispositivo: cuda


## Llama 3.2 lora (Fine-Tunned)

In [1]:
from unsloth import FastLanguageModel
import torch

2024-11-08 18:05:48.943105: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 18:05:48.955413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 18:05:48.971938: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 18:05:48.976660: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 18:05:48.988979: I tensorflow/core/platform/cpu_feature_guar

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [3]:
model_path = "../../Models/llama_3.2_FT_lora_4000_questions_short_answer_labels"
# model_path = "model_3.2_lora_4bits"

# Carregar o modelo e o tokenizador separadamente
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype = dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2024.10.6: Fast Llama patching. Transformers = 4.46.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU. Max memory: 3.712 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.6. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.10.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

# Dataset TeleQnA for Inference

In [1]:
import json

# Path to the TeleQnA processed question in JSON file
rel17_100_questions_path = r"../../Files/rel17_100_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_100_questions_path, "r", encoding="utf-8") as file:
    rel17_100_questions = json.load(file)
print(len(rel17_100_questions))

100


In [ ]:
rel17_100_questions[0]

{'question': 'Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]',
 'option 1': 'eNB Configuration Transfer',
 'option 2': 'Downlink RAN Configuration Transfer',
 'option 3': 'Uplink RAN Configuration Transfer',
 'option 4': 'MME Configuration Transfer',
 'answer': 'option 3: Uplink RAN Configuration Transfer',
 'explanation': 'The NGAP procedure used for inter-system load balancing is Uplink RAN Configuration Transfer.',
 'category': 'Standards overview'}

# Import RAG Functions

In [5]:
from rag_functions import load_faiss_index, search_faiss_index, search_RAG, load_chunks

In [6]:
# Test RAG
query_text = "reception of a transparent l3 message in unacknowledged mode"
index_file_path = "../../Files/faiss_index.bin"
chunks_path = "../../Files/tspec_chunks_markdown.pkl"
top_k = 5

In [7]:
result = search_RAG(query_text, index_file_path, chunks_path, top_k)
print(result)


Information 1:
BSC.  
Collision cases are treated as specified in 3GPPTS44.006.  
If BTS has repeated the DISC frame N200 times, BTS sends a RELease
INDication and an ERRor INDication message to BSC (cf. 3GPPTS44.006).  
![](media/image7.png){width="3.65625in" height="1.2083333333333333in"}  
3.5 Transmission of a transparent L3-Message in acknowledged mode
-----------------------------------------------------------------  
This procedure is used by BSC to request the sending of a L3 message to
MS in acknowledged mode.  
BSC sends a DATA REQuest message to BTS. The message contains the
complete L3 message to be sent in acknowledged mode.  
![](media/image8.png){width="3.6979166666666665in" height="1.0625in"}  
3.6 Reception of a transparent L3-Message in acknowledged mode
--------------------------------------------------------------  
This procedure is used by BTS to indicate the reception of a L3 message
in acknowledged mode.  
BTS sends a DATA INDication message to BSC. The message 

# Accuracy evaluation

## Create prompt and Ask function for Llama 3.2 lora (With Fine-Tuning)

In [7]:
import torch
from unsloth.chat_templates import get_chat_template

def ask_llama_3_2_lora_RAG(model, tokenizer, question_data, top_k=5, index_file_path="../../Files/faiss_index.bin", chunks_path="../../Files/tspec_chunks_markdown.pkl"):
    """
    Function to generate an answer using the model based on the given question and options, 
    including relevant information from a RAG search and prompting a chain of thought.
    
    Parameters:
    - model: The language model loaded for inference.
    - tokenizer: The tokenizer configured with `get_chat_template`.
    - question_data: Dictionary containing the question and options.
    - top_k: Number of relevant chunks to retrieve from the search.
    - index_file_path: Path to the FAISS index file.
    - chunks_path: Path to the chunks file.

    Returns:
    - String: Model's generated response.
    """

    # Extract question and options
    question = question_data['question']
    options = [f"{key}: {value}" for key, value in question_data.items() if 'option' in key]
    
    question_search = (
        f"{question}\n" +
        " ".join(options) + " "
    )

    # Perform RAG search using the question to retrieve relevant information
    rag_results = search_RAG(question_search, index_file_path=index_file_path, chunks_path=chunks_path, top_k=top_k)

    # Create the prompt with Chain of Thought (CoT) instructions
    prompt = (
        f"Relevant Information:\n{rag_results}\n"
        f"Question: {question}\n"
        f"Options:\n" + "\n".join(options) + "\n"
        # "Think step by step and analyze the relevant information carefully, then choose the correct option.\n"
        "Think step by step and choose the correct option.\n"
        "You must respond in the format 'correct option: <X>', where <X> is the correct letter for the option."
    )

    # Create the input for the model
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate the response
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=2048,
        temperature=0.7,
        min_p=0.9,
        use_cache=True
    )

    # Clear memory
    del inputs
    torch.cuda.empty_cache()

    # Decode and return the model's output
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return response


In [19]:
# Example usage
model = FastLanguageModel.for_inference(model)  # Enable faster inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

question_data = {
    'question': 'Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]',
    'option 1': 'PBCH',
    'option 2': 'PCFICH',
    'option 3': 'PDSCH',
    'option 4': 'PHICH',
    'answer': 'option 2: PCFICH',
    'explanation': 'The physical control format indicator channel (PCFICH) informs the UE and the RN about the number of OFDM symbols used for the PDCCHs.',
    'category': 'Standards specifications'
}

llama_3_2_lora_response = ask_llama_3_2_lora_RAG(model, tokenizer, question_data)
print(llama_3_2_lora_response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Relevant Information:
Information 1:
of OFDM symbols of the PUSCH, including all OFDM symbols used for DMRS;  
\- for any OFDM symbol that carries DMRS of the PUSCH,
$M_{\text{sc}}^{\text{UCI}}\left( l \right) = 0$;  
\- for any OFDM symbol that does not carry DMRS of the PUSCH,
$M_{\text{sc}}^{\text{UCI}}\left( l \right) = M_{\text{sc}}^{\text{PUSCH}} - \ M_{\text{sc}}^{PT - RS}\left( l \right)$;  
\- $\alpha$ is configured by higher layer parameter *scaling*;  
\- $l_{0}$ is the symbol index of the first OFDM symbol that does not
carry DMRS of the PUSCH, after the first DMRS symbol(s), in the PUSCH
transmission.  
For CG-UCI transmission on PUSCH with UL-SCH, and if
*numberOfSlotsTBoMS* is present in the resource allocation table and the
value of *numberOfSlotsTBoMS* in the row indicated by the Time domain
resource assignment field in DCI is larger than 1, the number of coded
modulation symbols per layer fo

## Evaluate Question 

In [8]:
import re

def extract_option(answer):
    """
    Extract the option part from the answer string, removing all punctuation and converting to lowercase.
    
    Parameters:
    - answer: A string containing the answer in the format 'option X: ...'.

    Returns:
    - String: Extracted option (e.g., 'option 2'), or None if no match is found.
    """
    # Remove all punctuation and convert to lowercase
    cleaned_answer = re.sub(r'[^\w\s]', '', answer.lower())
    # Find all matches for the format "option X"
    matches = re.findall(r'option \d+', cleaned_answer)
    # Return the last match with stripped whitespace if any found, otherwise None
    return matches[-1].strip() if matches else None

In [9]:
def extract_response_after_assistant(response):
    """
    Extract the part of the response that comes after the 'assistant' marker.

    Parameters:
    - response: The complete response from the model.

    Returns:
    - String: The extracted relevant part of the response.
    """
    # Split the response based on the 'assistant' marker
    parts = response.split('assistant', 1)
    # Return the part after 'assistant' or the entire response if 'assistant' is not found
    return parts[1].strip() if len(parts) > 1 else response.strip()

In [10]:
def evaluate_model_response(model_response, question_data):
    """
    Compare the model's response with the correct answer from the question data.
    
    Parameters:
    - model_response: The response string generated by the model.
    - question_data: Dictionary containing the question, options, and the correct answer.

    Returns:
    - 1 if the response is correct, otherwise the extracted model option.
    """
    correct_option = extract_option(question_data['answer'])  # Extract correct option
    relevant_response = extract_response_after_assistant(model_response)  # Get relevant part of response
    model_option = extract_option(relevant_response)  # Extract model's option

    return 1 if model_option == correct_option else model_option  # Return 1 if correct, else model's option


In [23]:
question_data = {
    'question': 'Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]',
    'option 1': 'PBCH',
    'option 2': 'PCFICH',
    'option 3': 'PDSCH',
    'option 4': 'PHICH',
    'answer': 'option 2: PCFICH',
    'explanation': 'The physical control format indicator channel (PCFICH) informs the UE and the RN about the number of OFDM symbols used for the PDCCHs.',
    'category': 'Standards specifications'
}

In [24]:
evaluation_result = evaluate_model_response(llama_3_2_lora_response, question_data)
print(evaluation_result)

1


## Ask to model Llama 3.2 lora TeleQnA 100 question 

In [11]:
def evaluate_questions(model, tokenizer, questions):
    """
    Process all questions and return the model responses.
    
    Parameters:
    - model: The language model loaded for inference.
    - tokenizer: The tokenizer configured with `get_chat_template`.
    - questions: List of dictionaries containing question data, where each dictionary has:
        - 'question': A string representing the question to be asked to the model.
        - 'answer': A string representing the correct answer format (e.g., 'option 2: PCFICH').
        - 'response': A string that will contain the model's generated response to the question.
    
    Returns:
    - List: A list of dictionaries where each dictionary contains:
        - 'question': The question as a string.
        - 'answer': The correct answer as a string.
        - 'response': The model's generated response for that question.
    """
    
    responses = []
    total_questions = len(questions)
    
    for idx, question_data in enumerate(questions):
        response = ask_llama_3_2_lora_RAG(model, tokenizer, question_data)
        responses.append({
            "question": question_data['question'],
            "answer": question_data['answer'],
            "response": response
        })
        
        # Print progress
        print(f"Responded {idx + 1} of {total_questions} questions...")

    return responses

In [12]:
model = FastLanguageModel.for_inference(model)  # Enable faster inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Process all questions and get responses
responses_llama_3_2_lora = evaluate_questions(model, tokenizer, rel17_100_questions)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Responded 1 of 100 questions...
Responded 2 of 100 questions...
Responded 3 of 100 questions...
Responded 4 of 100 questions...
Responded 5 of 100 questions...
Responded 6 of 100 questions...
Responded 7 of 100 questions...
Responded 8 of 100 questions...
Responded 9 of 100 questions...
Responded 10 of 100 questions...
Responded 11 of 100 questions...
Responded 12 of 100 questions...
Responded 13 of 100 questions...
Responded 14 of 100 questions...
Responded 15 of 100 questions...
Responded 16 of 100 questions...
Responded 17 of 100 questions...
Responded 18 of 100 questions...
Responded 19 of 100 questions...
Responded 20 of 100 questions...
Responded 21 of 100 questions...
Responded 22 of 100 questions...
Responded 23 of 100 questions...
Responded 24 of 100 questions...
Responded 25 of 100 questions...
Responded 26 of 100 questions...
Responded 27 of 100 questions...
Responded 28 of 100 questions...
Responded 29 of 100 questions...
Responded 30 of 100 questions...
Responded 31 of 100

In [13]:
print(responses_llama_3_2_lora[1]['response'])

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Relevant Information:
Information 1:
| Key issue 15   | Solution \#18: | 7.18.2         | \-             |
| -- Supporting  | Support for HD |                |                |
| dynamic        | map dynamic    |                |                |
| information    | information    |                |                |
| for HD maps    |                |                |                |
+----------------+----------------+----------------+----------------+
| Key issue 16   | Solution \#14: | 7.14.2         | \-             |
| --             | Support for    |                |                |
| Enhancements   | enhancements   |                |                |
| to V2X group   | to V2X group   |                |                |
| management and | management and |                |                |
| group          | group          |                |                |
| communication  | communication  |          

## Save accuracy responses

In [2]:
def save_responses_to_json(responses, filename):
    """
    Save the model responses to a JSON file.
    
    Parameters:
    - responses: List of responses to save.
    - filename: Name of the JSON file.
    """
    
    with open(filename, "w") as json_file:
        json.dump(responses, json_file, indent=4)

In [15]:
# save_responses_to_json(responses_llama_3_2_lora,"../../Models_responses/Accuracy/llama_3.2_lora_short_answer_RAG_responses.json")

## Evaluate responses from Llama 3.2 lora

In [20]:
import json

# Load responses from the JSON file
with open("../Models_responses/Accuracy/llama_3.2_lora_short_answer_RAG_responses.json", "r") as file:
    responses_llama_3_2_lora = json.load(file)

# Print the loaded responses to verify
print("Responses loaded")
# for response in responses_llama_3_2_lora:
#     print(response)


Responses loaded


In [16]:
def evaluate_accuracy(responses_llama_3_2_lora):
    """
    Evaluate the model's responses and calculate accuracy.
    """
    correct_count = 0  # Track the number of correct responses
    none_count = 0  # Track the number of 'None' responses

    for index, question_data in enumerate(responses_llama_3_2_lora):
        evaluation_result = evaluate_model_response(question_data['response'], question_data)
        options = [f"{key}: {value}" for key, value in rel17_100_questions[index].items() if 'option' in key]

        if evaluation_result == 1:
            correct_count += 1  # Increment for correct response
        elif evaluation_result is None:
            # Print only responses that are None
            print("\nWrong Answer")
            print(f"Question {index + 1}: {question_data['question']}")
            print(f"Options:\n" + "\n".join(options) + "\n")
            print(f"Full model response:\n{question_data['response']}")
            print(f"Correct response: {question_data['answer']}")
            print("----------------------------------------------------------------------------------------")
            none_count += 1  # Increment for None response
        else:
            print("\nWrong Answer")
            print(f"Question {index + 1}: {question_data['question']}")
            print(f"Options:\n" + "\n".join(options) + "\n")
            print(f"Model response: {evaluation_result}")
            print(f"Correct response: {question_data['answer']}")
            print("----------------------------------------------------------------------------------------")

    # Calculate and print accuracy
    accuracy = correct_count / len(responses_llama_3_2_lora) * 100
    print(f"\nAccuracy: {accuracy:.2f}%")
    print(f"Total 'None' responses: {none_count}")
    print(f"'None' responses means that the model did not give an option")


In [17]:
evaluate_accuracy(responses_llama_3_2_lora)


Wrong Answer
Question 5: What does MINT stand for? [3GPP Release 17]
Options:
option 1: Maximum Internet Network Transmission
option 2: Minimization of Interruption Network Technology
option 3: Maximization of Internet Network Traffic
option 4: Minimization of Service Interruption
option 5: Maximum Internet Network Technology

Model response: option 2
Correct response: option 4: Minimization of Service Interruption
----------------------------------------------------------------------------------------

Wrong Answer
Question 8: What is a capability added in the V2X Application Enabler (VAE) layer? [3GPP Release 17]
Options:
option 1: V2X UE group management
option 2: V2X service discovery
option 3: Session-oriented services
option 4: Network monitoring by the V2X UE
option 5: Support for PC5 provisioning

Model response: option 5
Correct response: option 2: V2X service discovery
----------------------------------------------------------------------------------------

Wrong Answer
Ques

# RAGAS evaluation

## Create prompt with no option and Ask function for Llama 3.2 lora

In [9]:
from unsloth.chat_templates import get_chat_template

def ask_llama_3_2_RAG_no_options(model, tokenizer, question_data):
    """
    Function to generate an answer using the model based on the given question and options.
    
    Parameters:
    - model: The language model loaded for inference.
    - tokenizer: The tokenizer configured with `get_chat_template`.
    - question_data: Dictionary containing the question and options.

    Returns:
    - String: Model's generated response.
    """

    # Extract question and options
    question = question_data['question']
    # options = [f"{key}: {value}" for key, value in question_data.items() if 'option' in key]
    
    question_search = (
        f"{question}"
        # + " ".join(options) + " "
    )

    # Perform RAG search using the question to retrieve relevant information
    rag_results = search_RAG(question_search, index_file_path=index_file_path, chunks_path=chunks_path, top_k=top_k)

    # Create the prompt with the question and options
    prompt = (
        f"Relevant Information:\n{rag_results}\n"
        f"Question: {question}\n"
        # "Think step by step before answering, analyse and respond with a final answer in the format 'answer: <XXXXX>'."
        "Think step by step before answering and analyze the relevant information carefully, then respond with a final answer in the format 'answer: <XXXXX>'."
    )

    # Create the input for the model
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate the response
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=2048,
        use_cache=True,
        temperature=0.7,
        min_p=0.9
    )
    
    # Clear memory
    del inputs # Remove tensors to free memory
    torch.cuda.empty_cache()  # Release unused memory

    # Decode and return the model's output
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return response


In [11]:
# Example usage
model = FastLanguageModel.for_inference(model)  # Enable faster inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

question_data = {
    'question': 'Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]',
    'option 1': 'PBCH',
    'option 2': 'PCFICH',
    'option 3': 'PDSCH',
    'option 4': 'PHICH',
    'answer': 'option 2: PCFICH',
    'explanation': 'The physical control format indicator channel (PCFICH) informs the UE and the RN about the number of OFDM symbols used for the PDCCHs.',
    'category': 'Standards specifications'
}

llama_3_2_response_text = ask_llama_3_2_RAG_no_options(model, tokenizer, question_data)_no_options_in_search
print(llama_3_2_response_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Relevant Information:
Information 1:
| UE Category                                 |      | 11-12    |   |
+---------------------------------------------+------+----------+---+
| UE DL Category                              |      | ≥ 11     |   |
+---------------------------------------------+------+----------+---+
| Note 1: 2 symbols allocated to PDCCH for 20 |      |          |   |
| MHz, 15 MHz and 10 MHz channel BW; 3        |      |          |   |
| symbols allocated to PDCCH for 5 MHz and 3  |      |          |   |
| MHz; 4 symbols allocated to PDCCH for 1.4   |      |          |   |
| MHz. For subframe 1&6, only 2 OFDM symbols  |      |          |   |
| are allocated to PDCCH. For 256QAM          |      |          |   |
| reference channel 1 symbol is allocated.    |      |          |   |
|                                             |      |          |   |
| Note 2: Reference signal, synchronization  

In [3]:
def format_answer(answer):
    # Remove punctuation and convert to lowercase
    answer_no_punctuation = answer.translate(str.maketrans('', '', string.punctuation))
    return answer_no_punctuation.lower()

In [12]:
# import re
# import string

# def extract_answer(text):
#     # Verifica a presença de 'assistant'
#     assistant_match = re.search(r'assistant\s*(.*)', text, re.IGNORECASE | re.DOTALL)
#     if assistant_match:
#         assistant_text = assistant_match.group(1).strip()

#         # Verifica se existe 'answer:' dentro do texto após 'assistant'
#         answer_match = re.search(r'answer:\s*(.*)', assistant_text, re.IGNORECASE | re.DOTALL)
#         if answer_match:
#             return answer_match.group(1).strip()  # Retorna tudo após 'answer:'

#         # Se não encontrar 'answer:', retorna tudo após 'assistant'
#         return assistant_text

#     # Se não encontrar 'assistant', retorna None
#     return None


In [4]:
import re
import string

def extract_answer(text):
    # Check for the presence of 'assistant'
    assistant_match = re.search(r'assistant\s*(.*)', text, re.IGNORECASE | re.DOTALL)
    if assistant_match:
        # If 'assistant' is found, get the text that follows it
        assistant_text = assistant_match.group(1).strip()

        # Find all occurrences of 'answer:' and capture the text after the last one
        answer_matches = re.findall(r'answer:\s*(.*)', assistant_text, re.IGNORECASE | re.DOTALL)
        
        # Return the phrase after the last 'answer:' found
        return answer_matches[-1].strip() if answer_matches else assistant_text

    # Return None if 'assistant' is not found
    return None


In [5]:
def extract_option(text):
    # Find all occurrences of 'option X:' followed by text, where X can be any number
    option_matches = re.findall(r'option\s*\d+:\s*(.*)', text, re.IGNORECASE | re.DOTALL)
    
    # Return the text after the last 'option X:' found
    return option_matches[-1].strip() if option_matches else None

In [ ]:
extracted_answer = extract_answer(llama_3_2_response_text)
print(extracted_answer)

In [20]:
model_response = format_answer(extracted_answer)
print(model_response)

dmrs symbols are used to transmit information about the number of ofdm symbols used for the pdcchs on pdcch


In [21]:
question_data['answer']

'option 2: PCFICH'

In [25]:
correct_answer = format_answer(extract_option(question_data['answer']))
print(correct_answer)

pcfich


## Model for RAGAS evaluation

In [25]:
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [36]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    # model="llama-3.1-70b-versatile",
    model="llama3-70b-8192",
    # model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipykernel_123051/34686554.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/arimatea/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-11-08 18:08:27.369512: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from di

## Ask to model Llama 3.2 lora TeleQnA 100 question with no options

In [13]:
def evaluate_questions_no_options(model, tokenizer, questions):
    """
    Process all questions and return the model responses.
    
    Parameters:
    - model: The language model loaded for inference.
    - tokenizer: The tokenizer configured with `get_chat_template`.
    - questions: List of dictionaries containing question data, where each dictionary has:
        - 'question': A string representing the question to be asked to the model.
        - 'answer': A string representing the correct answer format (e.g., 'option 2: PCFICH').
        - 'response': A string that will contain the model's generated response to the question.
    
    Returns:
    - List: A list of dictionaries where each dictionary contains:
        - 'question': The question as a string.
        - 'answer': The correct answer as a string.
        - 'response': The model's generated response for that question.
    """
    
    responses = []
    total_questions = len(questions)
    
    for idx, question_data in enumerate(questions):
        response = ask_llama_3_2_RAG_no_options(model, tokenizer, question_data)
        responses.append({
            "question": question_data['question'],
            "answer": question_data['answer'],
            "response": response
        })
        
        # Print progress
        print(f"Responded {idx + 1} of {total_questions} questions...")

    return responses

In [14]:
model = FastLanguageModel.for_inference(model)  # Enable faster inference
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Process all questions and get responses
llama_3_2_responses_RAGAS = evaluate_questions_no_options(model, tokenizer, rel17_100_questions)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Responded 1 of 100 questions...
Responded 2 of 100 questions...
Responded 3 of 100 questions...
Responded 4 of 100 questions...
Responded 5 of 100 questions...
Responded 6 of 100 questions...
Responded 7 of 100 questions...
Responded 8 of 100 questions...
Responded 9 of 100 questions...
Responded 10 of 100 questions...
Responded 11 of 100 questions...
Responded 12 of 100 questions...
Responded 13 of 100 questions...
Responded 14 of 100 questions...
Responded 15 of 100 questions...
Responded 16 of 100 questions...
Responded 17 of 100 questions...
Responded 18 of 100 questions...
Responded 19 of 100 questions...
Responded 20 of 100 questions...
Responded 21 of 100 questions...
Responded 22 of 100 questions...
Responded 23 of 100 questions...
Responded 24 of 100 questions...
Responded 25 of 100 questions...
Responded 26 of 100 questions...
Responded 27 of 100 questions...
Responded 28 of 100 questions...
Responded 29 of 100 questions...
Responded 30 of 100 questions...
Responded 31 of 100

In [18]:
print(llama_3_2_responses_RAGAS[0]['question'])
print(extract_answer(llama_3_2_responses_RAGAS[0]['response']))
print(llama_3_2_responses_RAGAS[0]['answer'])

Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]
Inter-system load balancing procedure
option 3: Uplink RAN Configuration Transfer


In [19]:
(llama_3_2_responses_RAGAS[0]['response'])

"system\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nuser\n\nRelevant Information:\nInformation 1:\nF.3.2 3GPP AAA server behaviour\n-------------------------------  \nThe 3GPP AAA server may report its current load by including a Load AVP\nof type HOST in answer commands as described in IETFRFC8583\\[54\\].  \nThe 3GPP AAA server calculates its current load by implementation\nspecific means. For example, the 3GPP AAA server may take into account\nthe traffic over the STa interface or other interfaces, the level of\nusage of internal resources (e.g. CPU, memory), the access to external\nresources, etc.  \nThe 3GPP AAA server determines when to send Load AVPs of type HOST by\nimplementation specific means.  \nF.3.3 Trusted non 3GPP access network behaviour\n-----------------------------------------------  \nWhen performing next hop Diameter Agent selection for requests that are\nrouted based on realm, the Trusted non 3GPP access network may take into\naccount lo

In [28]:
# save_responses_to_json(llama_3_2_responses_RAGAS,"../../Models_responses/RAGAS/llama_3.2_lora_short_answer_RAG_responses_RAGAS_0.7_temperature.json")

## Build Dataset for Evaluation with RAGAS

In [9]:
import json

# Path to the TeleQnA processed question in JSON file
llama_3_2_responses_RAGAS_path = r"../../Models_responses/RAGAS/llama_3.2_lora_short_answer_RAG_responses_RAGAS.json"

# Load the TeleQnA data just release 17
with open(llama_3_2_responses_RAGAS_path, "r", encoding="utf-8") as file:
    llama_3_2_responses_RAGAS = json.load(file)
print(len(llama_3_2_responses_RAGAS))

100


In [10]:
from datasets import Dataset 

In [11]:
def transform_dataset(data):
    """Transform the dataset to the required format."""
    transformed_data = {
        'user_input': [],
        'response': [],
        'reference': []
    }

    for item in data:
        # print(f"\n{item['question']}\n{item['answer']}\n{item['response']}")
        question = item['question']
        model_response = format_answer(extract_answer(item['response']))
        correct_answer = format_answer(extract_option(item['answer']))
        # model_response = (extract_answer(item['response']))
        # correct_answer = (item['answer'])
        
        # Ensure model_response and correct_answer end with a period
        model_response = model_response.rstrip('.') + '.'
        correct_answer = correct_answer.rstrip('.') + '.'

        transformed_data['user_input'].append(question)
        transformed_data['response'].append(model_response)
        transformed_data['reference'].append(correct_answer)

    return transformed_data

In [12]:
# Transform the llama_3_2_responses_RAGAS dataset
# data_samples = transform_dataset(llama_3_2_responses_RAGAS[:20])
data_samples = transform_dataset(llama_3_2_responses_RAGAS)

# Create the dataset object
dataset = Dataset.from_dict(data_samples)

# Print to verify the structure
print(dataset)

Dataset({
    features: ['user_input', 'response', 'reference'],
    num_rows: 100
})


In [13]:
dataset[0]

{'user_input': 'Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]',
 'response': 'intersystem load balancing procedure.',
 'reference': 'uplink ran configuration transfer.'}

## Evaluate Llama 3.2 lora with RAGAS Metrics

### Using LLM to evaluate (Factual Correctness, Semantic similarity and Rubrics based criteria scoring)

In [14]:
from ragas import evaluate
from ragas.run_config import RunConfig
from ragas.metrics._factual_correctness import FactualCorrectness
from ragas.metrics import SemanticSimilarity
from ragas.metrics import RubricsScoreWithReference

In [15]:
factualCorrectness = FactualCorrectness()
semantiSimilarity = SemanticSimilarity()
rubrics = {
    "score1_description": "The response is incorrect, irrelevant, or does not align with the ground truth.",
    "score2_description": "The response partially matches the ground truth but includes significant errors, omissions, or irrelevant information.",
    "score3_description": "The response generally aligns with the ground truth but may lack detail, clarity, or have minor inaccuracies.",
    "score4_description": "The response is mostly accurate and aligns well with the ground truth, with only minor issues or missing details.",
    "score5_description": "The response is fully accurate, aligns completely with the ground truth, and is clear and detailed.",
}
rubricsScoreWithReference =  RubricsScoreWithReference(rubrics=rubrics)

In [38]:
score = evaluate(
    dataset,
    metrics=[
        factualCorrectness,
        semantiSimilarity,
        rubricsScoreWithReference,
    ],
    llm=llm,
    embeddings=embeddings,
    run_config = RunConfig(timeout=400, max_retries=20, max_wait=120,log_tenacity=False),
)
score.to_pandas()

Evaluating:   0%|          | 0/300 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[159]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[111]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[132]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[255]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[189]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could 

,user_input,response,reference,factual_correctness,semantic_similarity,rubrics_score_with_reference
0,Which NGAP procedure is used for inter-system ...,intersystem load balancing procedure.,uplink ran configuration transfer.,0.00,0.228364,1
1,What is covered by enhanced application layer ...,all of the above.,advanced v2x services.,NaN,0.113409,1
2,What does the Load-Balancing steering mode do?...,divides the traffic of a data flow into two po...,splits the traffic of a data flow across 3gpp ...,0.57,0.858245,3
3,What is the main objective of intent driven ma...,satisfy the new or updated intent.,to reduce the complexity of management for net...,0.00,0.084886,1
4,What does MINT stand for? [3GPP Release 17],minimization of service interruption.,minimization of service interruption.,0.00,1.000000,4
...,...,...,...,...,...,...
95,Which RRC state is the UE in when no RRC conne...,inactive.,rrcidle.,0.00,0.211736,1
96,How are the antenna elements placed on each an...,a rectangular arrangement.,in both the vertical and horizontal directions.,0.00,0.370383,2
97,What information may be provided to an emergen...,current dispatchable location if available.,both 1 and 2.,0.00,0.125532,2
98,What is the purpose of cross-network slice coo...,to avoid any interference in the slice communi...,to coordinate network slices in multiple 5g ne...,0.00,0.488470,2


In [ ]:
score

In [20]:
llama_3_2_evaluation_RAGAS_LLM = score.to_pandas()
# llama_3_2_evaluation_RAGAS_LLM.to_csv("../../Evaluations/RAGAS/LLM_evaluation/llama_3_2_lora_short_answer_RAG_evaluation_RAGAS_LLM.csv", index=False)

In [40]:
import pandas as pd
result = pd.read_csv("../../Evaluations/RAGAS/LLM_evaluation/llama_3_2_lora_short_answer_RAG_evaluation_RAGAS_LLM.csv")

In [41]:
result

,user_input,response,reference,factual_correctness,semantic_similarity,rubrics_score_with_reference
0,Which NGAP procedure is used for inter-system ...,intersystem load balancing procedure.,uplink ran configuration transfer.,0.00,0.228364,1
1,What is covered by enhanced application layer ...,all of the above.,advanced v2x services.,NaN,0.113409,1
2,What does the Load-Balancing steering mode do?...,divides the traffic of a data flow into two po...,splits the traffic of a data flow across 3gpp ...,0.57,0.858245,3
3,What is the main objective of intent driven ma...,satisfy the new or updated intent.,to reduce the complexity of management for net...,0.00,0.084886,1
4,What does MINT stand for? [3GPP Release 17],minimization of service interruption.,minimization of service interruption.,0.67,1.000000,4
...,...,...,...,...,...,...
95,Which RRC state is the UE in when no RRC conne...,inactive.,rrcidle.,0.00,0.211736,1
96,How are the antenna elements placed on each an...,a rectangular arrangement.,in both the vertical and horizontal directions.,0.00,0.370383,2
97,What information may be provided to an emergen...,current dispatchable location if available.,both 1 and 2.,0.00,0.125532,2
98,What is the purpose of cross-network slice coo...,to avoid any interference in the slice communi...,to coordinate network slices in multiple 5g ne...,0.00,0.488470,2


### No need LLM to evaluate (BleuScore, RougeScore, ExactMatch and StringPresence)

In [23]:
from ragas.metrics import BleuScore, RougeScore, ExactMatch, StringPresence
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/arimatea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
bleuScore = BleuScore()
rougeScore = RougeScore()
exactMatch = ExactMatch()
stringPresence = StringPresence()

In [27]:
score = evaluate(
    dataset,
    metrics=[
        bleuScore,
        rougeScore,
        exactMatch,
        stringPresence
    ],
    llm=llm,
    embeddings=embeddings
)
score.to_pandas()

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

/home/arimatea/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/arimatea/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/arimatea/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoo

,user_input,response,reference,bleu_score,rouge_score,exact_match,string_present
0,Which NGAP procedure is used for inter-system ...,intersystem load balancing procedure.,uplink ran configuration transfer.,1.218332e-231,0.000000,0.0,0.0
1,What is covered by enhanced application layer ...,all of the above.,advanced v2x services.,1.218332e-231,0.000000,0.0,0.0
2,What does the Load-Balancing steering mode do?...,divides the traffic of a data flow into two po...,splits the traffic of a data flow across 3gpp ...,1.469925e-01,0.425532,0.0,0.0
3,What is the main objective of intent driven ma...,satisfy the new or updated intent.,to reduce the complexity of management for net...,8.676910e-232,0.133333,0.0,0.0
4,What does MINT stand for? [3GPP Release 17],minimization of service interruption.,minimization of service interruption.,1.000000e+00,1.000000,1.0,1.0
...,...,...,...,...,...,...,...
95,Which RRC state is the UE in when no RRC conne...,inactive.,rrcidle.,1.531972e-231,0.000000,0.0,0.0
96,How are the antenna elements placed on each an...,a rectangular arrangement.,in both the vertical and horizontal directions.,4.739132e-232,0.000000,0.0,0.0
97,What information may be provided to an emergen...,current dispatchable location if available.,both 1 and 2.,1.164047e-231,0.000000,0.0,0.0
98,What is the purpose of cross-network slice coo...,to avoid any interference in the slice communi...,to coordinate network slices in multiple 5g ne...,1.268852e-231,0.166667,0.0,0.0


In [28]:
score

{'bleu_score': 0.0656, 'rouge_score': 0.2433, 'exact_match': 0.0600, 'string_present': 0.0700}

In [29]:
llama_3_2_evaluation_RAGAS_no_LLM = score.to_pandas()
# llama_3_2_evaluation_RAGAS_no_LLM.to_csv("../../Evaluations/RAGAS/No_LLM_evaluation/llama_3_2_lora_short_answer_RAG_evaluation_RAGAS_no_LLM.csv", index=False)

In [30]:
import pandas as pd
result = pd.read_csv("../../Evaluations/RAGAS/No_LLM_evaluation/llama_3_2_lora_short_answer_RAG_evaluation_RAGAS_no_LLM.csv")

In [31]:
result

,user_input,response,reference,bleu_score,rouge_score,exact_match,string_present
0,Which NGAP procedure is used for inter-system ...,intersystem load balancing procedure.,uplink ran configuration transfer.,1.218332e-231,0.000000,0.0,0.0
1,What is covered by enhanced application layer ...,all of the above.,advanced v2x services.,1.218332e-231,0.000000,0.0,0.0
2,What does the Load-Balancing steering mode do?...,divides the traffic of a data flow into two po...,splits the traffic of a data flow across 3gpp ...,1.469925e-01,0.425532,0.0,0.0
3,What is the main objective of intent driven ma...,satisfy the new or updated intent.,to reduce the complexity of management for net...,8.676910e-232,0.133333,0.0,0.0
4,What does MINT stand for? [3GPP Release 17],minimization of service interruption.,minimization of service interruption.,1.000000e+00,1.000000,1.0,1.0
...,...,...,...,...,...,...,...
95,Which RRC state is the UE in when no RRC conne...,inactive.,rrcidle.,1.531972e-231,0.000000,0.0,0.0
96,How are the antenna elements placed on each an...,a rectangular arrangement.,in both the vertical and horizontal directions.,4.739132e-232,0.000000,0.0,0.0
97,What information may be provided to an emergen...,current dispatchable location if available.,both 1 and 2.,1.164047e-231,0.000000,0.0,0.0
98,What is the purpose of cross-network slice coo...,to avoid any interference in the slice communi...,to coordinate network slices in multiple 5g ne...,1.268852e-231,0.166667,0.0,0.0
